In [1]:
# Clean data for modeling

pitch = pd.read_csv('../data/mlb-pitches.csv', index_col = [0])
pitch = pitch[['player_name', 'p_throws', 'pitch_type','release_speed', 'release_spin_rate', 'spin_axis', 
               'pfx_-x', 'pfx_z', 'bauer_units', 'effective_speed', 'release_pos_x', 'release_pos_z', 
               'release_extension', 'release_pos_y', 'plate_-x', 'plate_x', 'plate_z', 'type', 'balls','strikes', 
               'pitch_count', 'stand', 'bb_type', 'description', 'events', 'hit_distance_sc', 'launch_speed',
               'launch_angle', 'launch_speed_angle', 'estimated_ba_using_speedangle', 
               'estimated_woba_using_speedangle', 'woba_value', 'woba_denom', 'babip_value', 'iso_value', 
               'at_bat_number', 'pitch_number', 'inning', 'inning_topbot', 'home_score', 'away_score', 
               'post_home_score', 'post_away_score', 'on_1b', 'on_2b', 'on_3b', 'outs_when_up', 
               'delta_run_exp']].copy()

#Rename some columns
col_dict = {
    'release_speed': 'velo',
    'release_spin_rate': 'spin_rate',
    'launch_speed': 'exit_velo',
    'estimated_ba_using_speedangle': 'xba',
    'estimated_woba_using_speedangle': 'xwobacon'
}
pitch.rename(columns = col_dict, inplace = True)
pitch.to_csv('../data/model-pitches.csv')

In [2]:
# Run Expectany Table

# 2010-2015 Run Expectancy
# matrix = [[0, 0, 0, 0, 0.481], [1, 0, 0, 0, 0.859], [0, 1, 0, 0, 1.100], [1, 1, 0, 0, 1.437], 
#           [0, 0, 1, 0, 1.350], [1, 0, 1, 0, 1.784], [0, 1, 1, 0, 1.964], [1, 1, 1, 0, 2.292], 
#           [0, 0, 0, 1, 0.254], [1, 0, 0, 1, 0.509], [0, 1, 0, 1, 0.664], [1, 1, 0, 1, 0.884], 
#           [0, 0, 1, 1, 0.950], [1, 0, 1, 1, 1.130], [0, 1, 1, 1, 1.376], [1, 1, 1, 1, 1.541],
#           [0, 0, 0, 2, 0.098], [1, 0, 0, 2, 0.224], [0, 1, 0, 2, 0.319], [1, 1, 0, 2, 0.429], 
#           [0, 0, 1, 2, 0.353], [1, 0, 1, 2, 0.478], [0, 1, 1, 2, 0.580], [1, 1, 1, 2, 0.752]]

# 2019 Run Expectancy
matrix = [[0, 0, 0, 0, 0.544], [1, 0, 0, 0, 0.935], [0, 1, 0, 0, 1.147], [1, 1, 0, 0, 1.537], 
          [0, 0, 1, 0, 1.369], [1, 0, 1, 0, 1.759], [0, 1, 1, 0, 1.971], [1, 1, 1, 0, 2.362], 
          [0, 0, 0, 1, 0.298], [1, 0, 0, 1, 0.564], [0, 1, 0, 1, 0.713], [1, 1, 0, 1, 0.979], 
          [0, 0, 1, 1, 0.953], [1, 0, 1, 1, 1.219], [0, 1, 1, 1, 1.368], [1, 1, 1, 1, 1.634],
          [0, 0, 0, 2, 0.115], [1, 0, 0, 2, 0.242], [0, 1, 0, 2, 0.339], [1, 1, 0, 2, 0.467], 
          [0, 0, 1, 2, 0.391], [1, 0, 1, 2, 0.518], [0, 1, 1, 2, 0.615], [1, 1, 1, 2, 0.743]]

re = pd.DataFrame(matrix, columns = ['on_1b', 'on_2b', 'on_3b', 'outs_when_up', 're'])
re.to_csv('../data/run_expectancy_table.csv')

In [3]:
# wOBA by Count

woba_value = [['0-0', 0.310], ['0-1', 0.262], ['0-2', 0.196], ['1-0', 0.355], ['1-1', 0.293], ['1-2', 0.223], 
              ['2-0', 0.436], ['2-1', 0.352], ['2-2', 0.273], ['3-0', 0.622], ['3-1', 0.470], ['3-2', 0.384]]

woba_value = pd.DataFrame(woba_value, columns = ['pitch_count', 'count_woba_value'])
woba_value.to_csv('../data/count_woba_value.csv')

In [4]:
# Add RV to model-pitches.csv

data = pd.read_csv('../data/model-pitches.csv', index_col = [0])
data.dropna(subset = ['pitch_type', 'velo', 'spin_rate', 'pfx_-x', 
                      'release_extension', 'delta_run_exp'], inplace = True)

data['events'].replace(['sac_bunt', 'double_play', 'caught_stealing_2b', 'strikeout_double_play',
                        'other_out', 'sac_fly_double_play', 'pickoff_2b', 'pickoff_3b', 'triple_play'
                        'caught_stealing_home', 'pickoff_caught_stealing_2b', 'pickoff_3b', 
                        'sac_bunt_double_play', 'pickoff_caught_stealing_3b', 'pickoff_1b', 
                        'caught_stealing_3b', 'triple_play', 'caught_stealing_home'], 'field_out', inplace = True)
data['events'].replace(['catcher_interf'], 'field_error', inplace = True)

data['description'].replace(['called_strike', 'swinging_strike', 'swinging_strike_blocked', 'missed_bunt',
                             'foul'], 'strike', inplace = True)
data['description'].replace(['hit_by_pitch'], 'ball', inplace = True)

data['inning_topbot'] = data.inning_topbot.map({'Top': 0, 'Bot': 1})
data['on_1b'] = [1 if x > 1 else 0 for x in data['on_1b']]
data['on_2b'] = [1 if x > 1 else 0 for x in data['on_2b']]
data['on_3b'] = [1 if x > 1 else 0 for x in data['on_3b']]

data['home_runs'] = data['post_home_score'] - data['home_score']
data['away_runs'] = data['post_away_score'] - data['away_score']
data['runs'] = data['home_runs'] + data['away_runs']

#re = pd.read_csv('../data/run_expectancy_table.csv', index_col = [0])
data = pd.merge(data, re, how = 'left', on = ['on_1b', 'on_2b', 'on_3b', 'outs_when_up'])

data['re_end_state'] = data['delta_run_exp'] + data['re']
data['re24'] = data['re_end_state'] - data['re'] + data['runs']

#count_woba = pd.read_csv('../data/count_woba_value.csv', index_col = [0])
data = pd.merge(data, woba_value, how = 'left', on = ['pitch_count'])
data['count_woba_diff'] = round(-data['count_woba_value'].diff(1), 3)
data['count_woba_after'] = data['count_woba_value'] + data['count_woba_diff']
data['count_woba_diff'].fillna(0, inplace = True)
data['count_woba_after'].fillna(0, inplace = True)
#data['events'].fillna('na', inplace = True)
#data['count_woba_after'] = [0 if x != 'na' else y for (x, y) in zip(data['events'], data['count_woba_after'])]
data['count_woba_diff'] = data['count_woba_after'] - data['count_woba_value']
data['xRV'] = round((data['count_woba_diff']) / 1.15, 3)

data.to_csv('../data/model-pitches-rv.csv')

pd.set_option('max_columns', None)
print(data.shape)
data.head()

(705403, 58)


,player_name,p_throws,pitch_type,velo,spin_rate,spin_axis,pfx_-x,pfx_z,bauer_units,effective_speed,release_pos_x,release_pos_z,release_extension,release_pos_y,plate_-x,plate_x,plate_z,type,balls,strikes,pitch_count,stand,bb_type,description,events,hit_distance_sc,exit_velo,launch_angle,launch_speed_angle,xba,xwobacon,woba_value,woba_denom,babip_value,iso_value,at_bat_number,pitch_number,inning,inning_topbot,home_score,away_score,post_home_score,post_away_score,on_1b,on_2b,on_3b,outs_when_up,delta_run_exp,home_runs,away_runs,runs,re,re_end_state,re24,count_woba_value,count_woba_diff,count_woba_after,xRV
0,"Smith, Will",L,FF,92.3,2330.0,148.0,-8.28,16.56,25.243770,92.8,1.40,6.80,6.5,54.03,0.69,-0.69,2.83,X,1,2,1-2,R,ground_ball,hit_into_play,field_out,13.0,95.2,-13.0,2.0,0.174,0.158,0.0,1.0,0.0,0.0,61,4,9,0,5,0,5,0,0,0,0,2,-0.073,0,0,0,0.115,0.042,-0.073,0.223,-0.223,0.000,-0.194
1,"Smith, Will",L,SL,80.6,2254.0,315.0,9.24,5.76,27.965261,81.2,1.60,6.64,6.4,54.15,0.71,-0.71,2.62,S,1,1,1-1,R,NaN,strike,NaN,108.0,75.3,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,3,9,0,5,0,5,0,0,0,0,2,-0.027,0,0,0,0.115,0.088,-0.027,0.293,-0.070,0.223,-0.061
2,"Smith, Will",L,CU,75.5,1940.0,328.0,7.80,-6.12,25.695364,75.2,1.46,6.88,6.2,54.34,0.04,-0.04,2.46,S,1,0,1-0,R,NaN,strike,NaN,157.0,83.5,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,2,9,0,5,0,5,0,0,0,0,2,-0.020,0,0,0,0.115,0.095,-0.020,0.355,-0.062,0.293,-0.054
3,"Smith, Will",L,CU,75.0,2017.0,330.0,8.28,-8.28,26.893333,74.5,1.53,6.83,5.9,54.61,-2.10,2.10,3.89,B,0,0,0-0,R,NaN,ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,1,9,0,5,0,5,0,0,0,0,2,0.016,0,0,0,0.115,0.131,0.016,0.310,0.045,0.355,0.039
4,"Smith, Will",L,FF,91.2,2281.0,143.0,-7.56,15.36,25.010965,90.9,1.49,6.66,6.3,54.15,0.31,-0.31,2.80,X,1,0,1-0,L,ground_ball,hit_into_play,field_out,9.0,93.3,-18.0,2.0,0.100,0.090,0.0,1.0,0.0,0.0,60,2,9,0,5,0,5,0,0,0,0,1,-0.189,0,0,0,0.298,0.109,-0.189,0.355,-0.045,0.310,-0.039


In [5]:
data.events.value_counts()

field_out         82861
strikeout         41807
single            24848
walk              14978
double             7815
home_run           5892
field_error        1291
triple              667
wild_pitch            5
passed_ball           1
stolen_base_2b        1
Name: events, dtype: int64

In [6]:
data.dropna(subset = ['events'], inplace = True)
print(data.shape)
data.head()

(180166, 58)


,player_name,p_throws,pitch_type,velo,spin_rate,spin_axis,pfx_-x,pfx_z,bauer_units,effective_speed,release_pos_x,release_pos_z,release_extension,release_pos_y,plate_-x,plate_x,plate_z,type,balls,strikes,pitch_count,stand,bb_type,description,events,hit_distance_sc,exit_velo,launch_angle,launch_speed_angle,xba,xwobacon,woba_value,woba_denom,babip_value,iso_value,at_bat_number,pitch_number,inning,inning_topbot,home_score,away_score,post_home_score,post_away_score,on_1b,on_2b,on_3b,outs_when_up,delta_run_exp,home_runs,away_runs,runs,re,re_end_state,re24,count_woba_value,count_woba_diff,count_woba_after,xRV
0,"Smith, Will",L,FF,92.3,2330.0,148.0,-8.28,16.56,25.243770,92.8,1.40,6.80,6.5,54.03,0.69,-0.69,2.83,X,1,2,1-2,R,ground_ball,hit_into_play,field_out,13.0,95.2,-13.0,2.0,0.174,0.158,0.0,1.0,0.0,0.0,61,4,9,0,5,0,5,0,0,0,0,2,-0.073,0,0,0,0.115,0.042,-0.073,0.223,-0.223,0.00,-0.194
4,"Smith, Will",L,FF,91.2,2281.0,143.0,-7.56,15.36,25.010965,90.9,1.49,6.66,6.3,54.15,0.31,-0.31,2.80,X,1,0,1-0,L,ground_ball,hit_into_play,field_out,9.0,93.3,-18.0,2.0,0.100,0.090,0.0,1.0,0.0,0.0,60,2,9,0,5,0,5,0,0,0,0,1,-0.189,0,0,0,0.298,0.109,-0.189,0.355,-0.045,0.31,-0.039
6,"Smith, Will",L,SL,80.3,2328.0,304.0,3.96,2.16,28.991283,80.8,1.81,6.55,6.2,54.30,0.69,-0.69,0.83,S,0,2,0-2,L,NaN,strike,strikeout,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,59,3,9,0,5,0,5,0,0,0,0,0,-0.152,0,0,0,0.544,0.392,-0.152,0.196,0.114,0.31,0.099
9,"Gsellman, Robert",R,SI,94.3,1982.0,221.0,16.20,10.56,21.018028,94.4,-1.20,6.05,6.5,53.98,0.26,-0.26,1.78,X,0,2,0-2,R,popup,hit_into_play,field_out,140.0,75.3,65.0,3.0,0.000,0.000,0.0,1.0,0.0,0.0,58,3,8,1,5,0,5,0,0,0,0,2,-0.061,0,0,0,0.115,0.054,-0.061,0.196,0.114,0.31,0.099
12,"Gsellman, Robert",R,SL,90.8,2052.0,215.0,0.36,5.04,22.599119,90.9,-1.48,5.95,6.4,54.14,-0.98,0.98,1.12,S,0,2,0-2,R,NaN,strike,strikeout,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,57,6,8,1,5,0,5,0,0,0,0,1,-0.101,0,0,0,0.298,0.197,-0.101,0.196,0.114,0.31,0.099


In [7]:
data.events.value_counts()

field_out         82861
strikeout         41807
single            24848
walk              14978
double             7815
home_run           5892
field_error        1291
triple              667
wild_pitch            5
passed_ball           1
stolen_base_2b        1
Name: events, dtype: int64

In [8]:
re.head()

,on_1b,on_2b,on_3b,outs_when_up,re
0,0,0,0,0,0.544
1,1,0,0,0,0.935
2,0,1,0,0,1.147
3,1,1,0,0,1.537
4,0,0,1,0,1.369


In [9]:
woba_value.head()

,pitch_count,count_woba_value
0,0-0,0.310
1,0-1,0.262
2,0-2,0.196
3,1-0,0.355
4,1-1,0.293


In [10]:
mlb = pd.read_csv('../data/mlbraw2.csv', index_col = [0])
pd.set_option('max_columns', None)
print(mlb.shape)
mlb.head(10)

(8692, 107)


,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher_1,fielder_2_1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,des2,final_pitch_game,final_pitch_at_bat,runs_scored_on_pitch,bat_score_after,final_pitch_inning,bat_score_start_inning,bat_score_end_inning,cum_runs_in_inning,runs_to_end_inning,base_out_state,avg_re,next_base_out_state,next_avg_re,change_re,re24
pitch_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
FF,2020-08-01,93.4,1.42,6.34,"Urías, Julio",606466,628711,field_out,hit_into_play,NaN,NaN,NaN,NaN,5.0,"Ketel Marte grounds out, shortstop Corey Seager to first baseman Matt Beaty.",R,R,L,ARI,LAD,X,6.0,ground_ball,1,0,2020,0.22,1.58,-0.23,2.73,NaN,NaN,NaN,0,1,Bot,119.30,166.55,NaN,NaN,669257,NaN,NaN,-4.675694,-135.773799,-6.874367,3.868106,33.537863,-11.299569,3.41,1.56,21.0,82.7,-6.0,91.9,2489.0,5.7,630100,628711,669257,607461,571970,621458,608369,592626,572041,605141,54.79,0.153,0.141,0.00,1.0,0.0,0.0,2.0,5,2,4-Seam Fastball,0,0,0,0,0,0,0,0,Standard,Standard,158.0,-0.021,-0.291,field_out,0,1,0,0,0,0,1,0,1,"0 outs, _ _ _",0.504171,"1 outs, _ _ _",0.279230,-0.224941,-0.224941
CH,2020-08-01,85.7,1.44,5.93,"Urías, Julio",545121,628711,walk,blocked_ball,NaN,NaN,NaN,NaN,14.0,Ildemaro Vargas walks.,R,R,L,ARI,LAD,B,NaN,NaN,3,2,2020,1.34,0.85,0.02,0.49,NaN,NaN,NaN,1,1,Bot,NaN,NaN,NaN,NaN,669257,NaN,NaN,-6.020264,-124.380348,-8.175028,15.179642,26.595959,-21.796241,3.36,1.63,NaN,NaN,NaN,84.9,1940.0,6.1,630100,628711,669257,607461,571970,621458,608369,592626,572041,605141,54.44,NaN,NaN,0.70,1.0,0.0,0.0,NaN,6,6,Changeup,0,0,0,0,0,0,0,0,Standard,Standard,128.0,0.024,0.226,walk,0,1,0,0,0,0,1,0,1,"1 outs, _ _ _",0.279230,"1 outs, 1b _ _",0.589391,0.310161,0.310161
FF,2020-08-01,94.2,1.08,6.24,"Urías, Julio",516782,628711,single,hit_into_play,NaN,NaN,NaN,NaN,5.0,Starling Marte singles on a line drive to left fielder Joc Pederson. Ildemaro Vargas to 2nd.,R,R,L,ARI,LAD,X,7.0,line_drive,3,1,2020,0.12,1.63,-0.26,2.44,NaN,NaN,545121.0,1,1,Bot,55.04,84.42,NaN,NaN,669257,NaN,NaN,-3.717263,-136.925868,-7.652756,2.381237,33.446765,-10.212867,3.41,1.56,275.0,88.0,21.0,93.1,2425.0,6.0,630100,628711,669257,607461,571970,621458,608369,592626,572041,605141,54.54,0.367,0.347,0.90,1.0,1.0,0.0,3.0,7,5,4-Seam Fastball,0,0,0,0,0,0,0,0,Infield shift,Standard,156.0,0.035,0.364,single,0,1,0,0,0,0,1,0,1,"1 outs, 1b _ _",0.589391,"1 outs, 1b 2b _",0.891192,0.301801,0.301801
FF,2020-08-01,94.6,1.49,6.15,"Urías, Julio",500871,628711,single,hit_into_play,NaN,NaN,NaN,NaN,5.0,Eduardo Escobar singles on a ground ball to center fielder AJ Pollock. Ildemaro Vargas scores. Starling Marte to 2nd.,R,R,L,ARI,LAD,X,8.0,ground_ball,0,2,2020,0.42,1.54,-0.22,2.45,NaN,545121.0,516782.0,1,1,Bot,121.26,123.88,NaN,NaN,669257,NaN,NaN,-5.326476,-137.505209,-7.269481,6.544901,34.162442,-11.197421,3.30,1.50,78.0,87.7,4.0,93.2,2517.0,5.8,630100,628711,669257,607461,571970,621458,608369,592626,572041,605141,54.73,0.393,0.373,0.90,1.0,1.0,0.0,2.0,8,3,4-Seam Fastball,0,0,0,0,0,1,1,0,Standard,Standard,156.0,0.096,1.147,single,0,1

In [11]:
#data.to_csv('../data/model-pitches-rv.csv')